HATE DETECTION : H2H

In [5]:
import pandas as pd
df = pd.read_csv("train.csv")

In [6]:
df

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
159566,ffe987279560d7ff,""":::::And for the second time of asking, when ...",0,0,0,0,0,0
159567,ffea4adeee384e90,You should be ashamed of yourself \n\nThat is ...,0,0,0,0,0,0
159568,ffee36eab5c267c9,"Spitzer \n\nUmm, theres no actual article for ...",0,0,0,0,0,0
159569,fff125370e4aaaf3,And it looks like it was actually you who put ...,0,0,0,0,0,0


In [7]:
import torch
import numpy as np
Y = []
for i in df.values:
    Y.append(torch.tensor(np.array(i[2:],dtype = np.float32)))
Y = torch.stack(Y)
print(Y)
Y.shape

tensor([[0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        ...,
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.]])


torch.Size([159571, 6])

EMBEDDINGS OF TWEETS

In [ ]:
import pickle
import pandas as pd
import json
from tqdm import tqdm
from sentence_transformers import SentenceTransformer
model_sen = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')


In [5]:
X = []
for i in tqdm(df["comment_text"]):
    j = torch.tensor(model_sen.encode(i))
    X.append(j)
X = torch.stack(X)

100%|██████████| 159571/159571 [8:54:03<00:00,  4.98it/s]   


In [6]:
#saving the embeddings for future use
with open('X.pkl', 'wb') as f:
    pickle.dump(X, f)

print("Stacked tensor saved to stacked_tensor.pkl")

Stacked tensor saved to stacked_tensor.pkl


In [1]:
import torch
import pickle

# Load the embeddings from the pickle file
with open('X.pkl', 'rb') as f:
    X = pickle.load(f)
X = X.unsqueeze(1)
# Print the loaded tensor
print(X)


tensor([[[ 0.0610, -0.0387, -0.0026,  ..., -0.0268, -0.0665, -0.0274]],

        [[-0.0121, -0.0163, -0.0072,  ...,  0.0131, -0.0092, -0.0323]],

        [[ 0.0751,  0.0372, -0.0001,  ...,  0.0099, -0.0469, -0.0610]],

        ...,

        [[ 0.0199,  0.1087,  0.0155,  ..., -0.0486, -0.0388,  0.0154]],

        [[ 0.0585, -0.0155, -0.0265,  ...,  0.0055, -0.0276,  0.0078]],

        [[ 0.0355,  0.0207, -0.0079,  ...,  0.0050,  0.0335, -0.0203]]])


In [4]:
X.shape

torch.Size([159571, 1, 768])

In [8]:
# combing X and Y to form a dataset
import tensorflow as tf
dataset = tf.data.Dataset.from_tensor_slices((X,Y))
dataset = dataset.cache()
dataset = dataset.shuffle(1000)
dataset = dataset.batch(16)
dataset = dataset.prefetch(5)

In [10]:
train = dataset.take(int(len(dataset)*.7))
val = dataset.skip(int(len(dataset)*.7)).take(int(len(dataset)*.2))
test = dataset.skip(int(len(dataset)*.9)).take(int(len(dataset)*.1))

In [11]:
train_generator = train.as_numpy_iterator()
train_generator.next()

(array([[[ 0.06369204, -0.00168109, -0.02053482, ...,  0.00918948,
          -0.06721765, -0.02267014]],
 
        [[ 0.05358709,  0.07470992,  0.01954894, ...,  0.03144461,
          -0.06003745, -0.00468988]],
 
        [[-0.02651252, -0.01105592,  0.00860522, ..., -0.01385898,
          -0.05161578,  0.00274984]],
 
        ...,
 
        [[ 0.06946471,  0.00547499, -0.03057263, ..., -0.00957406,
          -0.00618989, -0.04088716]],
 
        [[ 0.08229987, -0.02437766, -0.00373299, ..., -0.01593228,
          -0.06455202,  0.00978255]],
 
        [[ 0.03014052,  0.04527967, -0.00141046, ...,  0.01193245,
          -0.02542548, -0.02536843]]], dtype=float32),
 array([[0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0

MODEL

In [12]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Bidirectional, Dense, Activation
import torch
# Define the Keras model
model = Sequential([
    Bidirectional(LSTM(768, activation = 'tanh')),
    Dense(128),
    Activation('relu'),
    Dense(16),
    Activation('tanh'),
    Dense(6),
    Activation('sigmoid')
])
# model1(X_text_train.unsqueeze(1))

# Compile the model
# optimizer = tf.keras.optimizers.Adam(lr=0.01)
model.compile(loss='binary_crossentropy', optimizer= 'Adam')

# Print the model summary
model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ bidirectional (Bidirectional)   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation (Activation)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_1 (Activation)       │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_2 (Activation)       │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [13]:
history = model.fit(train, epochs=1, validation_data= val)

6981/6981 ━━━━━━━━━━━━━━━━━━━━ 1232s 174ms/step - loss: 0.0906 - val_loss: 0.0587


In [14]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ bidirectional (Bidirectional)   │ (None, 1536)           │     9,443,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       196,736 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation (Activation)         │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 16)             │         2,064 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_1 (Activation)       │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 6)              │           102 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_2 (Activation)       │ (None, 6)              │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 28,926,692 (110.35 MB)

 Trainable params: 9,642,230 (36.78 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 19,284,462 (73.56 MB)

PREDICTIONS

In [34]:
X_test = torch.tensor(model_sen.encode("I'll kill you")).unsqueeze(0).unsqueeze(0)
X_test

tensor([[[ 7.1044e-02,  4.8869e-02, -5.5874e-03,  4.4661e-03, -6.5701e-03,
           3.8937e-02, -3.0228e-02,  4.2765e-02, -1.2309e-02,  1.3392e-02,
           2.7514e-02,  2.2837e-02,  7.3720e-02,  5.7936e-02, -6.5131e-03,
           6.2170e-03,  1.0437e-02, -5.4102e-03,  5.1908e-02, -2.3087e-02,
          -5.3402e-03,  2.7886e-02, -6.2808e-03,  1.0456e-02,  1.0376e-02,
           1.3865e-02,  5.3587e-02,  2.5933e-02, -3.4686e-03, -3.3729e-02,
          -9.2243e-03, -4.4367e-02, -4.2160e-02,  2.2287e-02,  1.7971e-06,
          -4.2737e-02,  6.8327e-02, -1.1885e-03, -5.4989e-02,  1.0131e-01,
           4.8786e-03, -7.5937e-02, -2.4148e-02, -3.0298e-02,  1.2081e-02,
          -2.1161e-02,  9.9955e-03,  2.6894e-02,  2.1409e-02, -5.1982e-03,
          -8.0993e-03, -5.3718e-02,  2.8362e-02, -2.2426e-02,  7.7232e-03,
           1.9633e-02,  1.5691e-02,  3.1930e-02, -1.3382e-02, -3.5782e-03,
          -2.1913e-02, -7.3918e-03, -1.0127e-02,  4.6985e-02,  8.1481e-03,
           1.1517e-03,  5

In [35]:
Y_test = model.predict(X_test)
Y_test

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 212ms/step


array([[0.6462554 , 0.02151999, 0.22061332, 0.08073615, 0.2955304 ,
        0.00222081]], dtype=float32)

In [31]:
df.columns[2:]

Index(['toxic', 'severe_toxic', 'obscene', 'threat', 'insult',
       'identity_hate'],
      dtype='object')

In [37]:
test_batch_X, test_batch_Y = test.as_numpy_iterator().next()

In [41]:
predict_Y = (model.predict(test_batch_X) > 0.5).astype(int)
predict_Y

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 126ms/step


array([[0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [1, 0, 1, 0, 1, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0]])

In [42]:
test_batch_Y

array([[0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [1., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [1., 1., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.]], dtype=float32)

EVALUATING THE MODEL

In [50]:
from sklearn.metrics import precision_score, recall_score
precision = precision_score(test_batch_Y, predict_Y, average='micro')
recall = recall_score(test_batch_Y, predict_Y, average= 'micro')
accuracy = (predict_Y == test_batch_Y).astype(float).mean().item()
print("accuracy: ",accuracy)
print("recall: ",recall)
print("precision: ",precision)

accuracy:  0.9375
recall:  0.3333333333333333
precision:  0.5


SAVE MODEL

In [ ]:
from tensorflow.keras.models import save_model

save_model(model, 'hate_detection.keras')


In [ ]:
from tensorflow.keras.models import load_model

# Load the model
model = load_model('hate_detection.keras')

# Now you can use the loaded_model for predictions or further training


In [87]:
def evaluate_hate(tweet):
    eval_X = torch.tensor(model_sen.encode(tweet)).unsqueeze(0).unsqueeze(0)
    res = model.predict(eval_X)
    result = f'tweet: "{tweet}"\n'
    for i, col in enumerate(df.columns[2:]):
        result += f'{col}: {res[0][i]>0.5}\n'
    return result

TEST HERE

In [101]:
print("WELCOME TO HATE DETECTOR: H2H")
tweet = input("Enter the tweet to check")
print(evaluate_hate(tweet))

WELCOME TO HATE DETECTOR: H2H


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
tweet: "You fucker"
toxic: True
severe_toxic: False
obscene: True
threat: False
insult: True
identity_hate: False

